# Chapter 3 - Stack and Queues

## 3.1 Three in One
Describe how you could use a single array to implement three stacks.

In [1]:
class ThreeStackArray():
    def __init__(self, name_a, name_b, name_c):
        self.arr = []
        self.names = [name_a, name_b, name_c]
        self.a=-1
        self.b=-1
        self.c=-1
    def push(self,name,value):
        if name == self.names[0]:
            self.arr.insert(self.a+1,value)
            self.a+=1
            self.b+=1
            self.c+=1
        elif name == self.names[1]:
            self.arr.insert(self.b+1,value)
            self.b+=1
            self.c+=1
        elif name == self.names[2]:
            self.arr.insert(self.c+1,value)
            self.c+=1
        else:
            raise Exception(f'Push failed. Unknown stack name {name}')
    def pop(self,name):
        if name == self.names[0]:
            if self.a>=0:
                v = self.arr.pop(self.a)
                self.a-=1
                self.b-=1
                self.c-=1
                return v
            else:
                raise Exception(f'Pop Failed. Stack {name} has no more elements to pop')
        elif name == self.names[1]:
            if self.b-self.a>0:
                v = self.arr.pop(self.b)
                self.b-=1
                self.c-=1
                return v
            else:
                raise Exception(f'Pop Failed. Stack {name} has no more elements to pop')
        elif name == self.names[2]:
            if self.c-self.b>0:
                v = self.arr.pop(self.c)
                self.c-=1
                return v
            else:
                raise Exception(f'Pop Failed. Stack {name} has no more elements to pop')
        else:
            raise Exception(f'Pop failed. Unknown stack name {name}')
            
    def __repr__(self):
        a = f'{self.names[0]}: {self.arr[0:self.a+1]}'
        b = f'{self.names[1]}: {self.arr[self.a+1:self.b+1]}'
        c = f'{self.names[2]}: {self.arr[self.b+1:self.c+1]}'
        return '\n'.join([a,b,c])

In [2]:
stack = ThreeStackArray('A','B','C')
stack.push('A',1)
stack.push('A',2)
stack.push('A',3)
stack.push('A',4)
stack.push('B','a')
stack.push('B','b')
stack.push('B','c')
stack.push('C','hello')
stack.push('C','world')

print(stack)

pop_1 = [stack.pop('B'), stack.pop('A'), stack.pop('B')]
print('\nPopped', pop_1, '\n')

print(stack)

# Complexity
#  Time: push O(1) pop O(1)
#  Space: O(n+m+l)

A: [1, 2, 3, 4]
B: ['a', 'b', 'c']
C: ['hello', 'world']

Popped ['c', 4, 'b'] 

A: [1, 2, 3]
B: ['a']
C: ['hello', 'world']


## 3.2 Stack Min
How would you design a stack which, in addition to push and pop, has a function min which returns the minimum element? Push, pop and min should all operate in O(1) time.

In [3]:
# This is best done with a LinkedList version instead of using a
# pythonic list so we can store more values on each node
# (I'm not saying it couldn't be done with a pythonic list)
class Node():
    def __init__(self, value, next_node=None, curr_min=None):
        self.value=value
        self.curr_min=curr_min
        self.next_node=next_node
        
class Stack():
    def __init__(self):
        self.head = None
    def push(self,v):
        if self.head == None:
            self.head = Node(v, self.head, v)
        else:
            self.head = Node(v, self.head, min(v, self.head.curr_min))
    def pop(self):
        if self.head == None:
            raise Exception('Pop failed. Stack is empty')
        v = self.head.value
        self.head = self.head.next_node
        return v
    def peek(self):
        if self.head == None:
            return None
        return self.head.value
    def isEmpty(self):
        return self.head==None
    def min(self):
        return self.head.curr_min
    def __repr__(self,n=-1):
        curr_node = self.head
        if curr_node == None:
            return ""
        ret = [str(curr_node.value)]
        i=0
        while curr_node.next_node != None:
            if n>0 and i>=n:
                break
            curr_node = curr_node.next_node
            ret.append(str(curr_node.value))
            i+=1
        ret.reverse()
        return " <- ".join(ret)

In [4]:
stack = Stack()
stack.push(2)
stack.push(8)
stack.push(1)
print(stack, ':: Min is', stack.min())
pop_1 = stack.pop()
print('\nPopped',pop_1,'\n')
print(stack, ':: Min is', stack.min())

# Complexity
#  Time: push O(1) pop O(1) peek O(1) isEmpty O(1)
#  Space: O(n)

2 <- 8 <- 1 :: Min is 1

Popped 1 

2 <- 8 :: Min is 2


## 3.3 Stack of Plates

Imagine a (literal) stack of plates. If the stack gets too high, it might topple. Therefore, in real life, we would likely start a new stack when the previous stack exceeds some threshold. Implement a data structure `SetOfStacks` that mimics this. `SetOfStacks` should be composed of several stacks and should create a new stack once the previous one exceeds capacity.
`SetOfStacks.push()` and `SetOfStacks.pop()` should behave identically to a single stack (that is, `pop()` should return the same values as it would if there were just a single stack).

FOLLOW UP

Implement a function `popAt(int index)` which performs a pop operation on a specific sub-stack.

In [5]:
class SetOfStacks():
    def __init__(self, limit=5):
        self.stacks=[]
        self.limit=limit
    def push(self,value):
        if len(self.stacks) == 0 or len(self.stacks[-1])>=self.limit:
            s=[value]
            self.stacks.append(s)
        else:
            s=self.stacks[-1]
            s.append(value)
    def pop(self):
        if len(self.stacks) == 0:
            raise Exception('Pop failed. Stacks are empty.')
        else:
            s=self.stacks[-1]
            value=s.pop()
            if len(s)==0:
                self.stacks.pop()
            return value
    def popAt(self,i):
        s_idx=i//self.limit
        i_idx=i%self.limit
        
        try:
            s=self.stacks[s_idx]
            value=s.pop(i_idx)
        except:
            raise Exception(f'Pop failed. No item at index {i}.')
        
        # Clean up stacks
        if len(s)==0:
            self.stacks.pop()
        else:
            s_idx+=1
            while s_idx<len(self.stacks):
                p=s
                s=self.stacks[s_idx]
                p.append(s.pop(0))
                if len(s)==0:
                    self.stacks.pop()
                s_idx+=1
        
        return value
    def __repr__(self):
        return str(self.stacks)

In [6]:
my_stack = SetOfStacks()
my_stack.push('a')
my_stack.push('b')
my_stack.push('c')
my_stack.push('d')
my_stack.push('e')
my_stack.push('f')
my_stack.push('g')
print('Before:',my_stack)

pop1 = [my_stack.pop(),my_stack.pop(),my_stack.pop()]

print('\n  Popped:',pop1,'\n')
print('After:',my_stack)

my_stack.push('h')
my_stack.push('i')
my_stack.push('j')
my_stack.push('k')
my_stack.push('l')
my_stack.push('m')
my_stack.push('n')
print('Before:',my_stack)

pop4 = my_stack.popAt(2)
print('\n  Popped:',pop4,'\n')
print('After:',my_stack)

# Complexity
#  Time: push O(1) pop O(1) popAt O(log n) - the base of the log is the 'limit' (max size of each stack)
#  Space: O(n)

Before: [['a', 'b', 'c', 'd', 'e'], ['f', 'g']]

  Popped: ['g', 'f', 'e'] 

After: [['a', 'b', 'c', 'd']]
Before: [['a', 'b', 'c', 'd', 'h'], ['i', 'j', 'k', 'l', 'm'], ['n']]

  Popped: c 

After: [['a', 'b', 'd', 'h', 'i'], ['j', 'k', 'l', 'm', 'n']]


## 3.4 Queue via Stacks
Implement a `MyQueue` class which implements a queue using two stacks.

In [7]:
# Since this is intuitive with a pythonic list, I'll assume my elements are stored in singly linked list.
# I'll be using the Node/Stack structure from 3.2

class MyQueue():
    def __init__(self):
        self.new_stack = Stack()
        self.old_stack = Stack()
    def push(self, value):
        self.new_stack.push(value)
    def pop(self):
        self.__refresh_queue()
        return self.old_stack.pop()
    def peek(self):
        self.__refresh_queue()
        return self.old_stack.peek()
    def __refresh_queue(self):
        if self.old_stack.head == None:
            while(self.new_stack.head != None):
                self.old_stack.push(self.new_stack.pop())
    def __repr__(self):
        ret_o=[]
        curr_node = self.new_stack.head
        while curr_node != None:
            ret_o.append(str(curr_node.value))
            curr_node = curr_node.next_node
        ret_o.reverse()
        ret_n=[]
        curr_node = self.old_stack.head
        while curr_node != None:
            ret_n.append(str(curr_node.value))
            curr_node = curr_node.next_node
        return " <- ".join(ret_n+ret_o)

In [8]:
queue = MyQueue()
queue.push('a')
queue.push('b')
queue.push('c')
queue.push('d')
queue.pop()
queue.push('e')
print('Next:',queue.peek())
print(queue)

# Complexity
#  Time: push O(1) pop O(n) peek O(n)
#  Space: O(n)

Next: b
b <- c <- d <- e


## 3.5 Sort Stack
Write a program to sort a stack such that the smallest items are on the top. You can use an additional temporary stack, but you may not copy the elements into any other data structure (such as an array). The stack supports the following operations: `push`, `pop`, `peek`, and `isEmpty`.

In [9]:
# Again, I'll use the stack from 3.2 since this is trival with a list:
# sorted([3,6,2,5,1], reverse=True)

def sort_stack(stack):
    tmp_stack = Stack()
    while not stack.isEmpty():
        tmp = stack.pop()
        if tmp_stack.isEmpty() or tmp_stack.peek()<=tmp:
            tmp_stack.push(tmp)
        else:
            # Move larger items back to tmp_stack
            while not tmp_stack.isEmpty() and tmp_stack.peek()>tmp:
                stack.push(tmp_stack.pop())
            tmp_stack.push(tmp)
    
    # Copy the items back to stack
    while not tmp_stack.isEmpty():
        stack.push(tmp_stack.pop())

In [10]:
stack = Stack()
stack.push(3)
stack.push(6)
stack.push(2)
stack.push(5)
stack.push(1)
stack.push(7)
stack.push(4)
print('Before:',stack)
sort_stack(stack)
print('After:', stack)

# Complexity
#  Time: O(n log n)
#  Space: O(n)

Before: 3 <- 6 <- 2 <- 5 <- 1 <- 7 <- 4
After: 7 <- 6 <- 5 <- 4 <- 3 <- 2 <- 1


## 3.6 Animal Shelter
An animal shelter, which holds only dogs and cats, operates on a strictly "first in, first out" basis. People must adopt either the "oldest" (based on arrival time) of all animals at the shelter, or they can select whether they would prefer a dog or a cat (and will receive the oldest animal of that type). They cannot select which specific animal they would like. Create the data structures to maintain this system and implement operations such as `enqueue`, `dequeueAny`, `dequeueDog`, and `dequeueCat`. You may use the built-in `LinkedList` data structure.

In [11]:
class Node():
    def __init__(self, name, age, next_node=None):
        self.name=name
        self.age=age
        self.next_node=next_node

class AnimalShelter():
    def __init__(self):
        self.dogs=None
        self.cats=None
        self.dogs_need_sort=False
        self.cats_need_sort=False
    def dequeueDog(self):
        if self.dogs == None:
            raise Exception('No dogs!')
        if self.dogs_need_sort:
            self.dogs = self.__sort_by_age(self.dogs)
            self.dogs_need_sort=False
        oldest_dog = dict(name=self.dogs.name, age=self.dogs.age, pet='dog')
        self.dogs=self.dogs.next_node
        return oldest_dog
    def dequeueCat(self):
        if self.cats == None:
            raise Exception('No cats!')
        if self.cats_need_sort:
            self.cats = self.__sort_by_age(self.cats)
            self.cats_need_sort=False
        oldest_cat = dict(name=self.cats.name, age=self.cats.age, pet='cat')
        self.cats=self.cats.next_node
        return oldest_cat
    def dequeueAny(self):
        if self.cats == None:
            if self.dogs == None:
                raise Exception('No pets!')
            else:
                return self.dequeueDog()
        elif self.dogs == None:
            return self.dequeueCat()
        if self.dogs_need_sort:
            self.dogs = self.__sort_by_age(self.dogs)
            self.dogs_need_sort=False
        if self.cats_need_sort:
            self.cats = self.__sort_by_age(self.cats)
            self.cats_need_sort=False
        if self.cats.age>self.dogs.age:
            return self.dequeueCat()
        else:
            return self.dequeueDog()
    def __sort_by_age(self, s):
        tmp=None
        while s!=None:
            if tmp==None or tmp.age>=s.age:
                # Move older pets from s to tmp
                save = s.next_node
                s.next_node = tmp
                tmp = s
                s = save
            else:
                pet = s
                s = s.next_node
                # Move younger pets on tmp back to s
                while not tmp==None and tmp.age<pet.age:
                    save = tmp.next_node
                    tmp.next_node = s
                    s = tmp
                    tmp = save
                pet.next_node = tmp
                tmp = pet
        while tmp!=None:
            save = tmp.next_node
            tmp.next_node = s
            s = tmp
            tmp = save
        return s
    def enqueueCat(self, name, age):
        cat = Node(name, age, self.cats)
        self.cats = cat
        self.cats_need_sort=True
    def enqueueDog(self, name, age):
        dog = Node(name, age, self.dogs)
        self.dogs = dog
        self.dogs_need_sort=True
    def __repr__(self):
        d=[]
        dog=self.dogs
        while dog!=None:
            d.append(f'{dog.name}({dog.age})')
            dog=dog.next_node
        c=[]
        cat=self.cats
        while cat!=None:
            c.append(f'{cat.name}({cat.age})')
            cat=cat.next_node
        return 'Dogs: ' + ' -> '.join(d) + \
               '\nCats: ' + ' -> '.join(c)

In [12]:
ac = AnimalShelter()
ac.enqueueCat('Garfield',8)
ac.enqueueCat('Spots',3)
ac.enqueueCat('Gary',12)
ac.enqueueDog('Chase',3)
ac.enqueueDog('Boss',7)
ac.enqueueDog('Lassie',13)
ac.enqueueDog('Marshal',1)
ac.enqueueCat('Freckles',4)
print(f'Pre-Sort:\n{ac}')

cat = ac.dequeueCat()
print(f'\n -- Adopted: {cat}\n')
print(f'Cats Sorted:\n{ac}')

dog = ac.dequeueDog()
print(f'\n -- Adopted: {dog}\n')
print(f'Dogs Sorted:\n{ac}')

pet = ac.dequeueAny()
print(f'\n -- Adopted: {pet}\n')

ac.enqueueCat('Sprinkle',2)
print(f'Remaining Pets (added Sprinkle):\n{ac}')

pet = ac.dequeueAny()
print(f'\n -- Adopted: {pet}\n')
print(f'Sorted Both:\n{ac}')

# Complexity
#  Time: enqueue O(1) dequeue O(n log n)
#  Space: O(n)

Pre-Sort:
Dogs: Marshal(1) -> Lassie(13) -> Boss(7) -> Chase(3)
Cats: Freckles(4) -> Gary(12) -> Spots(3) -> Garfield(8)

 -- Adopted: {'name': 'Gary', 'age': 12, 'pet': 'cat'}

Cats Sorted:
Dogs: Marshal(1) -> Lassie(13) -> Boss(7) -> Chase(3)
Cats: Garfield(8) -> Freckles(4) -> Spots(3)

 -- Adopted: {'name': 'Lassie', 'age': 13, 'pet': 'dog'}

Dogs Sorted:
Dogs: Boss(7) -> Chase(3) -> Marshal(1)
Cats: Garfield(8) -> Freckles(4) -> Spots(3)

 -- Adopted: {'name': 'Garfield', 'age': 8, 'pet': 'cat'}

Remaining Pets (added Sprinkle):
Dogs: Boss(7) -> Chase(3) -> Marshal(1)
Cats: Sprinkle(2) -> Freckles(4) -> Spots(3)

 -- Adopted: {'name': 'Boss', 'age': 7, 'pet': 'dog'}

Sorted Both:
Dogs: Chase(3) -> Marshal(1)
Cats: Freckles(4) -> Spots(3) -> Sprinkle(2)
